# Milestone 5

In [ ]:
# Imports
import zipfile
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

import shap

# Specify global variables
RANDOM_STATE = 209

## 1 | Problem Statement

## 2 | EDA

In [ ]:
# Read in the final data
voter_gdf = gpd.read_file("../data/block_groups.geojson")

# Group the variables together
registered = ["total_reg"]
income = ["mean_hh_income"]
age = ["age_18_19", "age_20_24", "age_25_29", "age_30_34", "age_35_44", 
       "age_45_54", "age_55_64", "age_65_74", "age_75_84", "age_85over"]
gender = [ "gender_m", "gender_f", "gender_unknown"] 
party = ["party_npp", "party_dem", "party_rep", "party_lib", "party_grn", 
         "party_con", "party_ain", "party_scl", "party_oth"]
ethnicity1 = ["eth1_eur", "eth1_hisp", "eth1_aa",
                "eth1_esa", "eth1_oth", "eth1_unk"]
languages = ["lang_english", "lang_spanish", "lang_portuguese",
             "lang_chinese", "lang_italian", "lang_vietnamese", 
             "lang_other", "lang_unknown"]
predictors = [*registered, *income, *age, *gender, *party, *ethnicity1, *languages]

# Best predictors identified via LASSO and permutation test
predictors_subset = ["mean_hh_income", "total_reg", 
                     "lang_unknown", "gender_f", "party_dem", 
                     "eth1_hisp", "eth1_eur", "eth1_aa", "eth1_oth",
                     "age_20_24", "age_25_29", "age_30_34", "age_45_54",]

# Define X and y 
X = voter_gdf.copy()
y = voter_gdf["2020_turnout_pct"].copy()

## 3 | Modeling

## 4 | Results